In [1]:
!pip -q install timm==0.9.12 albumentations==1.4.0 opencv-python-headless -U 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.7 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This be

In [5]:
### START: CÁC KHAI BÁO CHÍNH - KHÔNG THAY ĐỔI ###
SEED = 25  # Số seed (Ban tổ chức sẽ công bố & thay đổi vào lúc chấm)
# Đường dẫn đến thư mục train
# (đúng theo cấu trúc gồm 4 thư mục cho 4 classes của ban tổ chức)
TRAIN_DATA_DIR_PATH = '/kaggle/input/ck-aio-hutech/oai_data/train'
# Đường dẫn đến thư mục test
TEST_DATA_DIR_PATH = '/kaggle/input/ck-aio-hutech/oai_data/test'
### END: CÁC KHAI BÁO CHÍNH - KHÔNG THAY ĐỔI ###


### START: CÁC THƯ VIỆN IMPORT ###
# Lưu ý: các thư viện & phiên bản cài đặt vui lòng để trong requirements.txt
#───────────────────────────────────────────────────────────────  
# 1) Imports & basic set‑up  
#───────────────────────────────────────────────────────────────  
import unicodedata
import os, gc, random, warnings  
from pathlib import Path  
from collections import Counter  

import numpy as np
import pandas as pd  
from PIL import Image  

import torch, torch.nn as nn  
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler  
from sklearn.model_selection import StratifiedKFold  

import albumentations as A  
from albumentations.pytorch import ToTensorV2  
import timm
warnings.filterwarnings("ignore")  
### END: CÁC THƯ VIỆN IMPORT ###


### START: SEEDING EVERYTHING - KHÔNG THAY ĐỔI ###
# Seeding nhằm đảm bảo kết quả sẽ cố định
# và không ngẫu nhiên ở các lần chạy khác nhau
# Set seed for random
random.seed(SEED)
# Set seed for numpy
np.random.seed(SEED)
# Set seed for torch
torch.manual_seed(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.manual_seed_all(SEED)
### END: SEEDING EVERYTHING - KHÔNG THAY ĐỔI ###


### START: ĐỊNH NGHĨA & CHẠY HUẤN LUYỆN MÔ HÌNH ###
#───────────────────────────────────────────────────────────────  
# 2) Paths  
#───────────────────────────────────────────────────────────────  
TRAIN_DIR = TRAIN_DATA_DIR_PATH 
TEST_DIR  = TEST_DATA_DIR_PATH

#───────────────────────────────────────────────────────────────  
# 3) Label map  
#───────────────────────────────────────────────────────────────  
label_map = {  
    unicodedata.normalize('NFC', "đông cô"): 0,  
    unicodedata.normalize('NFC', "tai mèo"): 1,  
    unicodedata.normalize('NFC', "tuyết khô"): 2  
}   
idx2cls = {v:k for k,v in label_map.items()}  
N_CLASS = len(label_map)  

#───────────────────────────────────────────────────────────────  
# 4) Collect image paths  
#───────────────────────────────────────────────────────────────  
def collect(root):  
    p,l = [],[]  
    for folder in os.listdir(root):  
        folder_norm = unicodedata.normalize('NFC', folder.lower().strip())  
        lbl = label_map[folder_norm]  
        for f in os.listdir(os.path.join(root, folder)):  
            p.append(os.path.join(root, folder, f)); l.append(lbl)  
    return p,l  

train_paths, train_labels = collect(TRAIN_DIR)  
test_paths  = [os.path.join(TEST_DIR, f) for f in sorted(os.listdir(TEST_DIR))]  

print("Train =", len(train_paths), " Test =", len(test_paths))  
print("Train distribution:", Counter(train_labels))  

#───────────────────────────────────────────────────────────────  
# 5) 5‑fold (use fold‑0 for validation)  
#───────────────────────────────────────────────────────────────  
skf = StratifiedKFold(5, shuffle=True, random_state=SEED)  
tr_idx, vl_idx = list(skf.split(train_paths, train_labels))[0]  

#───────────────────────────────────────────────────────────────  
# 6) Augmentations  (RandAugment removed)  
#───────────────────────────────────────────────────────────────  
IMG_SZ = 224  
train_tf = A.Compose([  
    A.RandomResizedCrop(IMG_SZ, IMG_SZ, scale=(0.6,1.0), ratio=(0.8,1.2)),  
    A.HorizontalFlip(p=0.5),  
    A.VerticalFlip(p=0.3),  
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.15,  
                       rotate_limit=20, border_mode=0, p=0.5),  
    A.HueSaturationValue(10,15,10, p=0.3),  
    A.RandomBrightnessContrast(0.15,0.15, p=0.4),  
    A.CoarseDropout(max_holes=4, max_height=IMG_SZ//10,  
                    max_width=IMG_SZ//10, p=0.3),  
    A.ToFloat(max_value=255.0), ToTensorV2()  
])  
val_tf = A.Compose([A.Resize(IMG_SZ, IMG_SZ),  
                    A.ToFloat(max_value=255.0), ToTensorV2()])

#───────────────────────────────────────────────────────────────  
# 7) Dataset / loader helpers  
#───────────────────────────────────────────────────────────────  
class ImgDS(Dataset):  
    def __init__(self, paths, labels=None, tf=None):  
        self.p, self.y, self.tf = paths, labels, tf  
    def __len__(self): return len(self.p)  
    def __getitem__(self, i):  
        img = Image.open(self.p[i]).convert("RGB")  
        img = self.tf(image=np.array(img))["image"]  
        if self.y is None: return img  
        return img, self.y[i]  

def make_loader(idxs, tf, bs, train=True):  
    paths  = [train_paths[i] for i in idxs]  
    labels = [train_labels[i] for i in idxs]  
    ds = ImgDS(paths, labels, tf)  
    if train:  
        sampler = WeightedRandomSampler(np.ones(len(labels)), len(labels))  
        return DataLoader(ds, bs, sampler=sampler,  
                          num_workers=2, pin_memory=True)  
    return DataLoader(ds, bs*2, shuffle=False,  
                      num_workers=2, pin_memory=True)  

BATCH = 32  
tr_loader = make_loader(tr_idx, train_tf, BATCH, True)  
vl_loader = make_loader(vl_idx, val_tf,   BATCH, False)  

#───────────────────────────────────────────────────────────────  
# 8) ViT‑small‑16 model  
#───────────────────────────────────────────────────────────────  
model = timm.create_model(  
    "vit_small_patch16_224",  
    pretrained=True,  
    num_classes=N_CLASS,  
    drop_path_rate=0.1  
).to(DEVICE)  

#───────────────────────────────────────────────────────────────  
# 9) Optimiser, loss, schedule  
#───────────────────────────────────────────────────────────────  
EPOCHS = 12 
LR  = 5e-5      # learning rate
WD  = 1e-4      # weight decay
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)    # label smoothing = 0.1
opt  = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)  
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=EPOCHS*len(tr_loader))  

os.makedirs("BEST_MODEL", exist_ok=True) # create a directory to save the best model

best_acc = 0  
for ep in range(1, EPOCHS+1):  
    model.train(); run=0  
    for xb,yb in tr_loader:  
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)  
        opt.zero_grad()  
        loss = criterion(model(xb), yb)  
        loss.backward(); opt.step(); sched.step()  
        run += loss.item()*yb.size(0)  
    tr_loss = run/len(tr_loader.dataset)  

    model.eval(); corr=tot=0  
    with torch.no_grad():  
        for xb,yb in vl_loader:  
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)  
            pred = model(xb).argmax(1)  
            corr += (pred==yb).sum().item(); tot += yb.size(0)  
    acc = 100*corr/tot
    best_acc=max(best_acc,acc)  
    print(f"Epoch {ep:02d}/{EPOCHS}  train‑loss: {tr_loss:.4f}  val‑acc: {acc:.2f}")  

print(f"Best val-accuracy: {best_acc:.2f}")  

#───────────────────────────────────────────────────────────────  
# 10) Quick fine‑tune on full data  
#───────────────────────────────────────────────────────────────  
print("\nQuick fine-tune on full data...")
full_loader = DataLoader(  
    ImgDS(train_paths, train_labels, train_tf),  
    batch_size=BATCH, shuffle=True, num_workers=2, pin_memory=True)  
opt = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=WD)  
for ep in range(1,6):  
    model.train(); run=0  
    for xb,yb in full_loader:  
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)  
        opt.zero_grad()  
        loss = criterion(model(xb), yb)  
        loss.backward(); opt.step()  
        run += loss.item()*yb.size(0)  
    print(f"[Full] Epoch {ep}/5  loss {run/len(full_loader.dataset):.4f}")  
    
#───────────────────────────────────────────────────────────────  
# 11) Inference  
#───────────────────────────────────────────────────────────────  
test_loader = DataLoader(ImgDS(test_paths, tf=val_tf),  
                         batch_size=BATCH*2, shuffle=False,  
                         num_workers=2, pin_memory=True)  
model.eval(); preds=[]  
with torch.no_grad():  
    for xb in test_loader:  
        preds.append(model(xb.to(DEVICE)).softmax(1).argmax(1).cpu().numpy())  
pred = np.concatenate(preds)  

print("\nPredicted distribution:")  
for i,c in enumerate(np.bincount(pred, minlength=N_CLASS)):  
    print(f"{i}: {idx2cls[i]:25} {c}")  
    
### END: ĐỊNH NGHĨA & CHẠY HUẤN LUYỆN MÔ HÌNH ###


### START: THỰC NGHIỆM & XUẤT FILE KẾT QUẢ RA CSV ###
#───────────────────────────────────────────────────────────────  
# 12) Submission  
#───────────────────────────────────────────────────────────────  
# os.makedirs("output", exist_ok=True)  
image_names = [os.path.splitext(os.path.basename(path))[0] for path in test_paths]   
submission_df = pd.DataFrame({  
    "image_name": image_names,  
    "label": pred  
})  
submission_df.to_csv("/kaggle/working/results.csv", index=False)   
print("\nFile results.csv saved!") 

### END: THỰC NGHIỆM & XUẤT FILE KẾT QUẢ RA CSV ###

Train = 1050  Test = 450
Train distribution: Counter({1: 350, 0: 350, 2: 350})
Epoch 01/12  train‑loss: 0.6373  val‑acc: 97.62
Epoch 02/12  train‑loss: 0.4004  val‑acc: 96.19
Epoch 03/12  train‑loss: 0.3629  val‑acc: 98.57
Epoch 04/12  train‑loss: 0.3484  val‑acc: 99.52
Epoch 05/12  train‑loss: 0.3321  val‑acc: 99.52
Epoch 06/12  train‑loss: 0.3276  val‑acc: 100.00
Epoch 07/12  train‑loss: 0.3180  val‑acc: 99.52
Epoch 08/12  train‑loss: 0.3198  val‑acc: 100.00
Epoch 09/12  train‑loss: 0.3179  val‑acc: 99.52
Epoch 10/12  train‑loss: 0.3133  val‑acc: 99.52
Epoch 11/12  train‑loss: 0.3103  val‑acc: 99.52
Epoch 12/12  train‑loss: 0.3126  val‑acc: 99.52
Best val-accuracy: 100.00

Quick fine-tune on full data...
[Full] Epoch 1/5  loss 0.3165
[Full] Epoch 2/5  loss 0.3103
[Full] Epoch 3/5  loss 0.3076
[Full] Epoch 4/5  loss 0.3059
[Full] Epoch 5/5  loss 0.3065

Predicted distribution:
0: đông cô                   152
1: tai mèo                   148
2: tuyết khô                 150

File resu